In [3]:
import ast


import pandas as pd
import numpy as np
import math

In [4]:
def read_file(file_name):
    mode='r'
    read_ops = ['dataset_read', 'blob_get', ] #'H5VLdataset_get'
    
    write_ops = ['dataset_write', 'blob_put']
    # 'file_open' # not used, dset has file info
    other_ops = ['object_open', 'dataset_open', 'dataset_create', 'object_close', 'dataset_close' ] 

    index = 0

    records = {}
    with open(file_name, mode) as f:
        for line in f:
            try:
                rec = ast.literal_eval(line)
                # print(data)
                    
                if any(op in rec['func_name'] for op in read_ops):
                    records[index] = rec
                    records[index]['operation'] = 'read'
                    index+=1
                elif any(op in rec['func_name'] for op in write_ops):
                    records[index] = rec
                    records[index]['operation'] = 'write'
                    index+=1
                elif any(op in rec['func_name'] for op in other_ops):
                    records[index] = rec
                    op_type = rec['func_name'].split('_')[1]
                    records[index]['operation'] = op_type
                    index+=1
                    
            except:
                print("Erro line:")
                print(line)
                break
    return records

def rec_to_df(records):
    df = pd.DataFrame.from_dict(records,orient='index')
    df.replace('/mnt/ssd/mtang11/','',regex=True, inplace=True)
    df.replace('molecular_dynamics_runs/stage0000/','',regex=True, inplace=True)
    
    # df['hash_id']= df['hash_id'].astype(str)
    # df['hash_id'] = map(lambda x: x.encode('base64','strict'), df['hash_id'])
    return df

def df_to_csc(df,file_name):
    df.hash_id=df.hash_id.astype('category').cat.codes
    out_csv=file_name.replace('.log','.csv')
    # print(df.columns)
    df.to_csv(out_csv)

def get_hash_id_list(df):
    hash_id_list = list(df.hash_id.unique())
    if -1 in hash_id_list:
        hash_id_list.remove(-1) # -1 is not valid hash_id
    return hash_id_list

def hash_id_groups(df,file_name):
    # observe hash_id and blob_access_index relations
    df['access_idx'] = df.index
    df = df.fillna(0)
    df['total_read_bytes'] = df['total_read_bytes'].astype(int)
    df['total_write_bytes'] = df['total_write_bytes'].astype(int)
    df = df[df['hash_id'] != -1] # remove hash_id = -1 --> not hashed    
    df = df[df['func_name'].str.contains("blob")] # select only blob operations
    
    grouped = df.groupby('hash_id', group_keys=True)

    out_txt=file_name.replace('.log','.txt')
    
    with open(out_txt, 'w') as f:
        for name, group in grouped:
            # tmp_df = grouped.get_group(idx)
            # tmp_df= tmp_df.sort_values(by=['blob_access_idx'],ascending=True)
            f.write(f'hash_id = {name} \n')
            rec = group.to_dict(orient='records')
            for r in rec:
                f.write(f'{r}\n')
            f.write('\n')

In [73]:
fsim='../tmp_outputs/prov-sim.log'
fagg='../tmp_outputs/prov-agg.log'


sim_rec = read_file(fsim)
agg_rec = read_file(fagg)

dfsim = rec_to_df(sim_rec)
dfagg = rec_to_df(agg_rec)

df_to_csc(dfsim,fsim)
df_to_csc(dfagg,fagg)

Erro line:




In [74]:
print(dfsim['blob_access_idx'].max())
print(dfagg['blob_access_idx'].max())

100
100


In [75]:
sim_hash_id = get_hash_id_list(dfsim)
agg_hash_id = get_hash_id_list(dfagg)

sim_hash_id = sim_hash_id.sort()
agg_hash_id = agg_hash_id.sort()

if sim_hash_id == agg_hash_id:
    print("Same")
else:
    print(f'agg_hash_id = {agg_hash_id}')
    print(f'agg_hash_id = {agg_hash_id}')

Same


In [5]:
from itertools import islice

# print(dfsim.head(10))
fsim='tmp_outputs/prov-sim.log'
sim_rec = read_file(fsim)
dfsim = rec_to_df(sim_rec)
dfsim1 = dfsim[dfsim['func_name'].str.contains("H5VLblob_put")] # select only blob operations

# dfsim2 = dfsim1.groupby(['blocklist'])['blocklist'].size()
# dfsim_idlist = dfsim1.groupby(['blocklist'])['hash_id'].apply(list)
# sim_count_list = []
sim_hashid_list = []
hashid_list = []
sim_blobidx_list = []
blobidx_list = []

# row = next(dfsim1.iterrows())[1]
tmp_logical_addr = dfsim1['logical_addr'].iloc[1]
hashid_list.append(dfsim1['hash_id'].iloc[0])
blobidx_list.append(dfsim1['blob_idx'].iloc[0])
# print(tmp_logical_addr)
count = 1

for index, row in islice(dfsim1.iterrows(), 1, None):
    if row['logical_addr'] == tmp_logical_addr:
        blobidx_list.append(row['blob_idx'])
        hashid_list.append(row['hash_id'])
    else:
        tmp_logical_addr = row['logical_addr']        
        sim_blobidx_list.append(blobidx_list)
        blobidx_list = [row['blob_idx'] ]

        # tmp_list = hashid_list.copy()
        sim_hashid_list.append(hashid_list)
        hashid_list= [row['hash_id']]

sim_blobidx_list.append(blobidx_list)
sim_hashid_list.append(hashid_list)

# print(sim_blobidx_list)
print(sum(len(l) for l in sim_blobidx_list))
# print(sim_hashid_list)
# print(dfsim_idlist)

40000


In [6]:
f2='tmp_outputs/prov-vfd-agg.log'
mode='r'

com_rec = {}
index=0

agg_blobidx_list = []
blobidx_list = []
agg_hashid_list = []
hashid_list = []
count = 0

with open(f2, mode) as f:
        for line in f:
            if "H5VLblob_get" in line:
                rec = ast.literal_eval(line)
                # print(data)
                # print(rec['blob_idx'])
                hashid_list.append(rec['hash_id'])
                blobidx_list.append(rec['blob_idx'])
                com_rec[index] = rec
                com_rec[index]['operation'] = 'read'
                index+=1
                count +=1

            else:
                if count != 0:
                    agg_blobidx_list.append(blobidx_list)
                    blobidx_list = []
                    agg_hashid_list.append(hashid_list)
                    hashid_list = []
                    count = 0

# count_list[0]+=1

# print(agg_count_list)
print(sum(len(l) for l in agg_blobidx_list))


if sim_blobidx_list == agg_blobidx_list:
    print("same blobidx_list")
else:
    print("different blobidx_list")
    for index, (first, second) in enumerate(zip(sim_blobidx_list, agg_blobidx_list)):
        if first != second:
            print(index, first, second)

if agg_hashid_list == sim_hashid_list:
    print("same hashid_list")
else:
    print("different hashid_list")
    for index, (first, second) in enumerate(zip(sim_hashid_list, agg_hashid_list)):
        if first != second:
            print(index, first, second)

# dfagg1 = rec_to_df(com_rec)
# dfsim1 = dfagg1[dfagg1['func_name'].str.contains("H5VLblob_get")] # select only blob operations
# dfagg1 = dfagg1.groupby(['blocklist'])['blocklist'].size()
# dfsim3 = dfsim1.groupby(['blocklist'])['hash_id'].apply(list)

40000
same blobidx_list
same hashid_list


In [7]:
from itertools import chain

agg_blobidx_list2 = []
blobidx_list = []
agg_hashid_list2 = []
hashid_list = []
count = 0
with open(f2, mode) as f:
        for line in f:
            if "H5VLblob_put" in line:
                rec = ast.literal_eval(line)
                # print(data)
                # print(rec['blob_idx'])
                hashid_list.append(rec['hash_id'])
                blobidx_list.append(rec['blob_idx'])
                com_rec[index] = rec
                com_rec[index]['operation'] = 'read'
                index+=1
                count +=1

            else:
                if count != 0:
                    agg_blobidx_list2.append(blobidx_list)
                    blobidx_list = []
                    agg_hashid_list2.append(hashid_list)
                    hashid_list = []
                    count = 0

print(len(agg_hashid_list2))



39482


In [13]:
flatten_agg_hashs = list(chain.from_iterable(agg_hashid_list))
flatten_agg_hashs2 = list(chain.from_iterable(agg_hashid_list2))
print(len(flatten_agg_hashs))
print(len(flatten_agg_hashs2))

if flatten_agg_hashs == flatten_agg_hashs2:
    print("same hashid_list")
else:
    print("different hashid_list")

diff_list = []
for i in range(0, len(flatten_agg_hashs)):
    if flatten_agg_hashs[i] != flatten_agg_hashs2[i]:
        diff_list.append([i, flatten_agg_hashs[i], flatten_agg_hashs2[i]])

print(diff_list)


80000
60116
different hashid_list


IndexError: list index out of range